In [1]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#cambiar el directorio de trabajo actual
os.chdir('/content/drive/MyDrive/datos')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
#lectura de datos por categoria
dataset=pd.read_csv("WineQT.csv")
dataset.head(1150)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,2
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,3
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,1592
1139,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6,1593
1140,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,1594
1141,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,1595


In [9]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, KBinsDiscretizer, MinMaxScaler

# Crear una copia del dataset original para aplicar los preprocesamientos
df_preprocessed_python = dataset.copy()

# 1. Aplicar LabelEncoder a la columna 'fixed acidity' (convirtiendo etiquetas en valores numéricos)
label_encoder = LabelEncoder()
df_preprocessed_python['fixed acidity'] = label_encoder.fit_transform(df_preprocessed_python['fixed acidity'])

# 2. Aplicar OneHotEncoder a la columna 'quality' (convertir la etiqueta en columnas binarias)
onehot_encoder = OneHotEncoder(sparse_output=False)  # Corregido de sparse a sparse_output
quality_encoded = onehot_encoder.fit_transform(df_preprocessed_python[['quality']])

# Añadir las columnas codificadas al dataframe (eliminando la columna original 'quality')
df_encoded = pd.DataFrame(quality_encoded, columns=[f'quality_{i}' for i in range(quality_encoded.shape[1])])
df_preprocessed_python = df_preprocessed_python.drop(columns=['quality'])
df_preprocessed_python = pd.concat([df_preprocessed_python, df_encoded], axis=1)

# 3. Discretización de las columnas numéricas 'alcohol', 'pH', 'residual sugar' en 5 categorías
discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
df_preprocessed_python[['alcohol', 'pH', 'residual sugar']] = discretizer.fit_transform(df_preprocessed_python[['alcohol', 'pH', 'residual sugar']])

# 4. Normalización de los datos (todas las columnas excepto 'Id')
scaler = MinMaxScaler()
df_preprocessed_python[df_preprocessed_python.columns[:-len(quality_encoded[0])]] = scaler.fit_transform(df_preprocessed_python[df_preprocessed_python.columns[:-len(quality_encoded[0])]])

# Mostrar el dataset preprocesado
# Mostrar el dataset preprocesado
print(df_preprocessed_python.head())

# Guardar el dataset preprocesado en un archivo CSV
df_preprocessed_python.to_csv('/content/drive/MyDrive/datos/dataset_preprocesado.csv', index=False)

#Las primeras columnas (como fixed acidity, volatile acidity, etc.) son normalizadas con valores entre 0 y 1.
#alcohol, pH, y residual sugar se encuentran discretizados en categorías.
#Las columnas quality_0, quality_1, etc., son las salidas del OneHotEncoder aplicadas a la columna original quality.



   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0       0.277778          0.397260         0.00             0.0   0.106845   
1       0.322222          0.520548         0.00             0.0   0.143573   
2       0.322222          0.438356         0.04             0.0   0.133556   
3       0.688889          0.109589         0.56             0.0   0.105175   
4       0.277778          0.397260         0.00             0.0   0.106845   

   free sulfur dioxide  total sulfur dioxide   density    pH  sulphates  \
0             0.149254              0.098940  0.567548  0.75   0.137725   
1             0.358209              0.215548  0.494126  0.25   0.209581   
2             0.208955              0.169611  0.508811  0.50   0.191617   
3             0.238806              0.190813  0.582232  0.25   0.149701   
4             0.149254              0.098940  0.567548  0.75   0.137725   

   alcohol        Id  quality_0  quality_1  quality_2  quality_3  quality_4  \
0